# Basic managed deployments with Docker and ACR 

In this article, we will deploy an inference server using a minimal custom Docker image and Azure Secure Container Registry. 

## Prerequisites

* To use Azure Machine Learning, you must have an Azure subscription. If you don't have an Azure subscription, create a free account before you begin. Try the [free or paid version of Azure Machine Learning](https://azure.microsoft.com/free/).

* Install and configure the [Python SDK v2](sdk/setup.sh).

* You must have an Azure resource group, and you (or the service principal you use) must have Contributor access to it.

* You must have an Azure Machine Learning workspace. 

* You must have an Azure Secure Container registry. One is created automatically created for a workspace without one upon first usage, however in this example we explicitly reference the container registry by name, so you need it beforehand. You can create one through the Azure Portal. 

## Initial set up

We will first get a handle to the workspace, which will be reused later as we deploy images. You must already have an existing Azure Secure Container Registry associated with the workspace.

In [1]:
subscription_id = '<YOUR_SUBSCRIPTION_ID>'
resource_group = '<YOUR_RESOURCE_GROUP>'
workspace = '<YOUR_WORKSPACE>'
container_registry_name = '<YOUR_CONTAINER_REGISTRY>'

In [2]:
import os
from azure.ml import MLClient
from azure.containerregistry import ContainerRegistryClient
from azure.ml.entities import ManagedOnlineDeployment, ManagedOnlineEndpoint
from azure.identity import DefaultAzureCredential
from random import randint

ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

## Basic Docker workflows

### Define deployment and container registry details

The name of the deployment, container registry, and container name are all required. We will create a new container using the name here, however, The endpoint name is optional, the code below will generate a random name likely to be unique within the region.

In [3]:
from random import randint
# Required
deployment_name = 'docker-basic'
container_name = 'docker-basic'
# Optional
endpoint_name = f'docker-basic-{randint(1e3,1e7)}'

The first image we will build is the Sklearn-0.24 Ubuntu 18.04 image from Azure. This image contains all of the dependencies required to score the model as well as an inference server. Our Dockerfile for this basic example is below: 

```Dockerfile 
FROM mcr.microsoft.com/azureml/sklearn-0.24.1-ubuntu18.04-py37-cpu-inference:latest
```

To begin, we will build the image and test a local deployment. If you're rebuilding, pass the `--no-cache` flag. We can build and test the image using Docker itself, however, with no scorin script or trained models, the container will fail immediately.

### Log in to ACR  and build an image locally

In [ ]:
!az login
!az acr login -n {container_registry_name}
!docker build {container_name} docker_basic/. 
!docker image ls 

### Push the locally-trained image to ACR

In [ ]:
!docker login {container_registry_name}.azurecr.io
!docker tag {container_name} {container_registry_name}.azurecr.io/storage-client
!docker push {container_registry_name}.azurecr.io/storage-client

### Build directly with the ACR CLI

In [ ]:
!az acr login -n {container_registry_name}
!az acr build --image {container_name} --registry {container_registry_name}  ./{deployment_name}

## Local Deployment

To deploy the inference server locally, we will proide the inference server with resources by setting our `Model`, `CodeConfiguration` and `Environment` in the ManagedOnlineDeployment YAML file. This file specifies the trained model `sklearn_regression_model.pkl1`, the scoring script under `score.py`, and the registry and repository of the image we built above.

```yaml 
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineDeployment.schema.json
name: deployment_name
endpoint_name: endpoint_name
model:
  path: sklearn_regression_model.pkl
code_configuration: 
  code: "."
  scoring_script: score.py
environment:
  image: container_registry_name.azurecr.io/docker-basic:latest
instance_type: Standard_F2s_v2
instance_count: 1
```

### Create an online endpoint

In [ ]:
endpoint = ManagedOnlineEndpoint(name=endpoint_name)
ml_client.online_endpoints.begin_create_or_update(endpoint, local=True)

### Import deployment YAML

We will import the YAML file for the deployment and update variables, however, the in your workloads the file can be directly loaded by passing the file path to the `.load` method of a `ManagedOnlineDeployment` object.

In [ ]:
import yaml
with open(f'{deployment_name}/deployment.yml','r') as f:
    deployment_yaml = yaml.safe_load(f)
deployment_yaml['name'] = deployment_name
deployment_yaml['endpoint_name'] = endpoint_name
deployment_yaml['environment']['image'] = f'{container_registry_name}.azurecr.io/{container_name}:latest'

Now we can deploy. First we create an endpoint and then a deployment. The code below shows two ways of configuring Azure Machine Learning entities using the Python SDK v2. We can provide configuration parameters either through arguments in the constructor, or through loading a YAML file. If you do not need to preprocess a YAML file, the `.load()` method enables you to pass a file path directly. 

In [ ]:
deployment = ManagedOnlineDeployment.load_from_dict(deployment_yaml)
deployment = ml_client.online_deployments.begin_create_or_update(deployment, local=True,)

### Check deployment logs

In [ ]:
!az ml online-deployment get-logs -n docker-basic -e {endpoint_name} --local

## Test the local endpoint

### Get token and scoring URL 

To test an endpoint, we need the scoring URI and an authentication key. When we called `.begin_create_or_update` above, the ml_client returned the endpoint object to us with metadata about the deployment, including the attribute `scoring_uri`. If we didn't have a reference to the endpoint, we would call `ml_client.online_endpoints.get(name=<ENDPOINT_NAME>)`. 

In [ ]:
auth_token = ml_client.online_endpoints.list_keys(endpoint_name).primary_key
endpoint = ml_client.online_endpoints.get(endpoint_name,local=True)
scoring_uri = endpoint.scoring_uri

### Score with REST

Online endpoints' scoring URIs end with `/score`. To check the aliveness of the endpoint without scoring data, a GET request can be made to the base URI.

In [ ]:
import requests

response = requests.get(scoring_uri[:-6])

To score data using REST, insert the auth token in the header, load the sample JSON file, and make a POST request to the scoring URI, which ends with `/score`. 

In [ ]:
import json 

with open('sample-request.json') as f:
    data = json.loads(f.read())
headers = {'Authorization' : f'Bearer {auth_token}'} 
response = requests.post(url=scoring_uri,
                        headers=headers,
                        data=json.dumps(data))

## Deploy to the Cloud
The scoring server can be deployed in the cloud with few configuration changes. Our Docker image is already built and available in the Azure Container Registry, and our deployment YAML file requires no changes. We first generate a new online endpoint name and proceed with similar steps as above. Note the removal of the `local=True` argument in `ml_client` methods.

### Prepare a new endpoint name and import YAML
There is no change to our YAML.

In [ ]:
endpoint_name = f'docker-basic-{randint(1e3,1e7)}'

import yaml
with open(os.path.join(deployment_name, 'deployment.yml'),'r') as f:
    deployment_yaml = yaml.safe_load(f)

deployment_yaml['endpoint_name'] = endpoint_name
deployment_yaml['environment']['image'] = f'{container_registry_name}.azurecr.io/{container_name}:latest'

### Create the endpoint and deployment

In [ ]:
endpoint = ManagedOnlineEndpoint(name=endpoint_name)
ml_client.online_endpoints.begin_create_or_update(endpoint)
deployment = ManagedOnlineDeployment.load_from_dict(deployment_yaml)
adeployment = ml_client.online_deployments.begin_create_or_update(deployment)

## Test and score the model

This time, we will score the model using the `.invoke` method.

In [ ]:
from pathlib import PurePath
json_file_path = PurePath(os.path.join(deployment_name, 'sample-request.json'))
ml_client.online_endpoints.invoke(endpoint_name=endpoint_name, deployment_name=deployment_name, request_file=json_file_path)

## YAML Files vs AML Python Objects and Entities

Azure YAML files are powerful, concise tools. The Python interface to Azure Machine Learning offers a complementary interface. The AML Python SDK v2. 

## Pre-loading dependencies in Docker images
Azure's flexible environment specifications make it trivially easy to deploy with pip, conda, or both, as well as custom code configurations. However in many circumstances it is necessary or advantageous preload as much as possible. Pushing dependencies to bulid time can help reduce deployment time or the size of the codebase, a prime use case for Docker. 

```dockerfile
FROM mcr.microsoft.com/azureml/minimal-ubuntu18.04-py37-cpu-inference:latest

RUN apt install python3, python3-pip
COPY requirements.txt /tmp/requirements.txt
RUN pip install -r /tmp/requirements.txt
```